# API Call
- Input file ('Resources/col_rename_df_updated.csv')
- Output file ('Resources/merged_col_rename_df_updated.csv')

In [1]:
import time
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import re

In [2]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")


In [3]:
col_rename_df_updated = pd.read_csv('Resources/col_rename_df_updated.csv')
col_rename_df_updated.head(5)

,Unnamed: 0,Name,Rank,Industry,Headquarters_State,Revenues,Year,Revenue_Percent_Change,Profits,Profits_Percent_Change,...,Profitable,Growth_in_Jobs,Sector,Revenues_($M),Profits_($M),Employees,Global_500,Change_in_Rank_(Full 1000),Change_in_Rank_(500 only),Headquarters_City
0,0,General Motors Corporation,1,Motor Vehicles and Parts,MI,"16,882,860",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Detroit
1,1,Ford Motor Company,2,Motor Vehicles and Parts,MI,"13,713,700",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dearborn
2,2,Exxon Corporation,3,Petroleum Refining,TX,"11,000,900",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Wal-Mart Stores, Inc.",4,General Merchandisers,AR,"9,362,700",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bentonville
4,4,AT&T Corp.,5,Telecommunications,NY,"7,960,900",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Generate list of cities that need coordinates

city_list = col_rename_df_updated['Headquarters_City']


city_list

0             Detroit
1            Dearborn
2                 NaN
3         Bentonville
4                 NaN
             ...     
13935        New York
13936    Redwood City
13937      Hartsville
13938     Santa Clara
13939      Menlo Park
Name: Headquarters_City, Length: 13940, dtype: object

In [5]:
# Empty lists for holding data
lat = []
lng = []
city = []
skip_list = []

In [6]:
# Record coordinates for each city
# This cell takes a LONG TIME to run

for i in city_list:
    if i in city or i is None or i in skip_list:
        pass
    else:
        try:
            location = geolocator.geocode(i)
            lat.append(location.latitude)
            lng.append(location.longitude)
            city.append(i)
            print(f'Adding coordinates for {i} to list')
            time.sleep(1)
        except:
            print(f'Skpping {i}')
            skip_list.append(i)
            time.sleep(1)

Adding coordinates for Detroit to list
Adding coordinates for Dearborn to list
Adding coordinates for nan to list
Adding coordinates for Bentonville to list
Adding coordinates for Armonk to list
Adding coordinates for Fairfield to list
Adding coordinates for Cincinnati to list
Adding coordinates for San Ramon to list
Adding coordinates for Palo Alto to list
Adding coordinates for Purchase to list
Adding coordinates for Schaumburg to list
Adding coordinates for New York to list
Adding coordinates for Bethesda to list
Adding coordinates for Hartford to list
Adding coordinates for Northbrook to list
Adding coordinates for Plano to list
Adding coordinates for Midland to list
Adding coordinates for Memphis to list
Adding coordinates for Chicago to list
Adding coordinates for Stamford to list
Adding coordinates for New Brunswick to list
Adding coordinates for Atlanta to list
Adding coordinates for Downers Grove to list
Adding coordinates for Fort Worth to list
Adding coordinates for Whitehou

In [7]:
# Create dataframe for merging
city_df = pd.DataFrame({
    'Headquarters_City': city,
    'Latitude': lat,
    'Longitude': lng
})

In [8]:
# Merge coordinates to the col_rename_df_updated
merged_col_rename_df_updated = col_rename_df_updated.merge(city_df,on='Headquarters_City', how='left')
merged_col_rename_df_updated

,Unnamed: 0,Name,Rank,Industry,Headquarters_State,Revenues,Year,Revenue_Percent_Change,Profits,Profits_Percent_Change,...,Sector,Revenues_($M),Profits_($M),Employees,Global_500,Change_in_Rank_(Full 1000),Change_in_Rank_(500 only),Headquarters_City,Latitude,Longitude
0,0,General Motors Corporation,1,Motor Vehicles and Parts,MI,"16,882,860",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Detroit,42.331551,-83.046640
1,1,Ford Motor Company,2,Motor Vehicles and Parts,MI,"13,713,700",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dearborn,42.322260,-83.176315
2,2,Exxon Corporation,3,Petroleum Refining,TX,"11,000,900",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.314475,11.048029
3,3,"Wal-Mart Stores, Inc.",4,General Merchandisers,AR,"9,362,700",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bentonville,36.372854,-94.208817
4,4,AT&T Corp.,5,Telecommunications,NY,"7,960,900",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.314475,11.048029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13935,13935,KKR,496,Securities,NY,NaN,2023,-72.2%,NaN,-118%,...,Financials,"$7,273",$-841,"4,150",no,-358.0,-358.0,New York,40.712728,-74.006015
13936,13936,Equinix,497,Real estate,CA,NaN,2023,9.5%,NaN,40.8%,...,Financials,"$7,263",$704,"12,097",no,-10.0,-10.0,Redwood City,37.486324,-122.232523
13937,13937,Sonoco Products,498,"Packaging, Containers",SC,NaN,2023,29.7%,NaN,NaN,...,Materials,"$7,251",$466,"22,000",no,58.0,NaN,Hartsville,42.247015,-77.693881
13938,13938,ServiceNow,499,Computer Software,CA,NaN,2023,22.9%,NaN,41.3%,...,Technology,"$7,245",$325,"20,433",no,32.0,NaN,Santa Clara,37.233325,-121.684635


In [9]:
# Generate list of states that need coordinates
state_list = []


for i in range(len(merged_col_rename_df_updated)):
    if merged_col_rename_df_updated['Headquarters_City'][i] == None:
        item = merged_col_rename_df_updated['Headquarters_State'][i]
        state_list.append(item)


In [10]:
# Generate empty lists to hold data
lat = []
lng = []
state = []

In [11]:
# Record coordinates for each state, when city is unavailable
for i in state_list:
    if i in state or i is None:
        pass
    else:
        try:
            location = geolocator.geocode(i)
            lat.append(location.latitude)
            lng.append(location.longitude)
            state.append(i)
            print(f'Adding coordinates for {i} to list')
            time.sleep(1)
        except:
            print(f'Skpping {i}')

In [12]:
# Generate dataframe of state coordinates
state_df = pd.DataFrame({
    'Headquarters_State': state,
    'Latitude': lat,
    'Longitude': lng
})

In [13]:
# Add coordinates to the merged dataframe
for i in range(len(merged_col_rename_df_updated)):
    if merged_col_rename_df_updated['Headquarters_City'][i] == None:
        merged_col_rename_df_updated['Latitude'][i] = state_df['Latitude'].loc[state_df['Headquarters_State'] == merged_col_rename_df_updated['Headquarters_State'][i]]
        merged_col_rename_df_updated['Longitude'][i] = state_df['Longitude'].loc[state_df['Headquarters_State'] == merged_col_rename_df_updated['Headquarters_State'][i]]

In [14]:
merged_col_rename_df_updated

,Unnamed: 0,Name,Rank,Industry,Headquarters_State,Revenues,Year,Revenue_Percent_Change,Profits,Profits_Percent_Change,...,Sector,Revenues_($M),Profits_($M),Employees,Global_500,Change_in_Rank_(Full 1000),Change_in_Rank_(500 only),Headquarters_City,Latitude,Longitude
0,0,General Motors Corporation,1,Motor Vehicles and Parts,MI,"16,882,860",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Detroit,42.331551,-83.046640
1,1,Ford Motor Company,2,Motor Vehicles and Parts,MI,"13,713,700",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dearborn,42.322260,-83.176315
2,2,Exxon Corporation,3,Petroleum Refining,TX,"11,000,900",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.314475,11.048029
3,3,"Wal-Mart Stores, Inc.",4,General Merchandisers,AR,"9,362,700",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bentonville,36.372854,-94.208817
4,4,AT&T Corp.,5,Telecommunications,NY,"7,960,900",1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.314475,11.048029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13935,13935,KKR,496,Securities,NY,NaN,2023,-72.2%,NaN,-118%,...,Financials,"$7,273",$-841,"4,150",no,-358.0,-358.0,New York,40.712728,-74.006015
13936,13936,Equinix,497,Real estate,CA,NaN,2023,9.5%,NaN,40.8%,...,Financials,"$7,263",$704,"12,097",no,-10.0,-10.0,Redwood City,37.486324,-122.232523
13937,13937,Sonoco Products,498,"Packaging, Containers",SC,NaN,2023,29.7%,NaN,NaN,...,Materials,"$7,251",$466,"22,000",no,58.0,NaN,Hartsville,42.247015,-77.693881
13938,13938,ServiceNow,499,Computer Software,CA,NaN,2023,22.9%,NaN,41.3%,...,Technology,"$7,245",$325,"20,433",no,32.0,NaN,Santa Clara,37.233325,-121.684635


In [15]:
# Export data
merged_col_rename_df_updated.to_csv('Resources/merged_col_rename_df_updated.csv')